# Setup

In [ ]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import datasets
    root_path = '.'
except ModuleNotFoundError:
    !git clone https://github.com/cgpotts/cs224u/
    !pip install -r cs224u/requirements.txt
    root_path = 'dsp'

In [ ]:
!pip uninstall dsp-ml
!pip install --no-cache git+https://github.com/stanfordnlp/dsp

Found existing installation: dsp-ml 0.1.5
Uninstalling dsp-ml-0.1.5:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/dsp/*
    /usr/local/lib/python3.10/dist-packages/dsp_ml-0.1.5.dist-info/*
Proceed (Y/n)? Y
  Successfully uninstalled dsp-ml-0.1.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/stanfordnlp/dsp to /tmp/pip-req-build-k2kcax8r
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/dsp /tmp/pip-req-build-k2kcax8r
  Resolved https://github.com/stanfordnlp/dsp to commit 298fac8937fdf7326d1b85546ecd41f099ed33e2
  Preparing metadata (setup.py) ... done
  Created wheel for dsp-ml: filename=dsp_ml-0.1.5-py3-none-any.whl size=37803 sha256=5efee867b5c72d13142b4ddc7aa95bf205f88ceea3424d90f5aa16ba3574f95b
  Stored in directory: /tmp/pip-ephem-wheel-cache-c2qd7s6p/wheels/b9/cc/21/df1431394d64701ffb9d4a5a20afe701a19275c3090b73fd0e
Successfully built dsp-ml


In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached faiss_cpu-1.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)


In [ ]:
!pip install editdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=e979d6638afe1b215a1cd942fd02a4a300aa2da046d81ab23b5630e576eda694
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import cohere
from datasets import load_dataset
import openai
import os
import dsp
import pandas as pd
import editdistance

In [ ]:
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')

openai_key = '*****'

In [ ]:
lm = dsp.GPT3(model = "gpt-3.5-turbo", api_key=openai_key, model_type="chat")

dsp.settings.configure(lm=lm)

In [ ]:
train = pd.read_csv('train.tsv',sep='\t', names=['input', 'output'])

FileNotFoundError: ignored

In [ ]:
train_dsp = [dsp.Example(input=row['input'], output=row['output'])
                    for _, row in train.iterrows()]

In [ ]:
import pandas as pd
test = pd.read_csv('test.tsv',sep='\t', names=['input', 'output'])

In [ ]:
indices = [i for i, ti in enumerate(test['input']) if len(ti)<=300][:250]
test_short = test.iloc[indices].reset_index()

In [ ]:
test_short

,index,input,output
0,2,"public Request iterate(final Request req, fina...","public Request iterate(final Request req, fina..."
1,4,public static <T> TypeToken<T> getTypeToken(Ty...,public static <T> TypeToken<T> getTypeToken(Ty...
2,5,private void initPlugins() throws Exception { ...,private void initPlugins() throws Exception { ...
3,15,protected Properties getIntegrationProperties(...,protected Properties getIntegrationProperties(...
4,16,public void testShowStatsSelectNonStarFails() ...,public void testShowStatsSelectNonStarFails() ...
...,...,...,...
245,970,public CompilePanel(CompileOutputDisplay outpu...,public CompilePanel(CompileOutputDisplay outpu...
246,972,"public void init(IActionBars bars, IWorkbenchP...","public void init(IActionBars bars, IWorkbenchP..."
247,974,"public ServerPermission(String name, String ac...","public ServerPermission(String name, String ac..."
248,975,public FunctionImplementationType getFunctionI...,public FunctionImplementationType getFunctionI...


# Zero Shot

In [ ]:
num = 3
temperature = 0.5
Input = dsp.Type(
    prefix="Input:",
    desc="${the code to be revised}")

Output = dsp.Type(
    prefix="Output:",
    desc="${the revised code}",
    format=dsp.format_answers)

zero_shot_template = dsp.Template(
    instructions="Revise the code to have better coding style by renaming, removing redundancy, safety checking, etc. Don't include any comments.",
    input=Input(),
    output=Output())


In [ ]:
def calc_dist(i, completions, f=None):
  dist = 0
  same = False
  if f:
    f.write(str(len(completions.data)) + '\n')
  for ind in completions.data:
    if f:
      f.write(ind.output.replace('\n', ''))
      f.write('\n')
    tmp1 = ''.join(ind.output.strip().split())
    tmp2 = ''.join(test['output'][i].strip().split())
    d = editdistance.eval(tmp1, tmp2) / len(tmp2)
    dist += d
    if d == 0:
      same = True
  return (dist / num), same

In [ ]:
with open('zero_shot_results.txt', 'w') as f:
  num_test = 250

  for i in range(num_test):
    if i % 10 == 0:
      print(i)
    ex = dsp.Example(input=test_short['input'][i], demos=[])
    ex, completions = dsp.generate(zero_shot_template, n=1, temperature=temperature)(ex, stage='qa')
    _, _ = calc_dist(i, completions, f)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


# Few Shot

In [ ]:
num = 3
temperature = 0.5
Input = dsp.Type(
    prefix="Input:",
    desc="${the code to be revised}")

Output = dsp.Type(
    prefix="Output:",
    desc="${the revised code}",
    format=dsp.format_answers)

Rationale = dsp.Type(
    prefix="Rationale:",
    desc="${the problems in the provided code}"
)

few_shot_template = dsp.Template(
    instructions="Revise the code to have better coding style by renaming, removing redundancy, safety checking, etc. Don't include any comments.",
    input=Input(),
    output=Output())


In [ ]:
with open('few_shot_results.txt', 'w') as f:
  num_test = 250
  num_demos = 3
  for i in range(num_test):
    if i % 10 == 0:
      print(i)
    ex = dsp.Example(input=test_short['input'][i], demos=dsp.sample(train_dsp, k=num_demos))
    ex, completions = dsp.generate(few_shot_template, n=1, temperature=temperature)(ex, stage='qa')
    dist, same = calc_dist(i, completions, f)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


# KNN

In [ ]:
import random
from typing import Callable, Any
import numpy as np
import dsp
from dsp.utils import EM, F1, DPR_normalize, dotdict, has_answer, normalize_text
from dsp.utils.ann_utils import create_faiss_index
from faiss import write_index,read_index

@dsp.transformation
def custom_knn(train, cast):
    train_casted_to_vectorize = [cast(cur_elem) for cur_elem in train]

    vectorizer = dsp.settings.vectorizer
    all_vectors = vectorizer(train_casted_to_vectorize).astype(np.float32)

    index = create_faiss_index(
        emb_dim=all_vectors.shape[1], n_objects=len(train)
    )
    index.train(all_vectors)
    index.add(all_vectors)
    return vectorizer, index

@dsp.transformation
def knn_search(inp_example: dsp.Example, train, cast, vectorizer, index, k: int) -> list[dsp.Example]:
    inp_example_vector = vectorizer([cast(inp_example)])
    _, nearest_samples_idxs = index.search(inp_example_vector, k)
    train_sampled = [train[cur_idx] for cur_idx in nearest_samples_idxs[0]]
    return train_sampled

In [ ]:
index = read_index("knn.index")

In [ ]:
def cast_naive_get_only_code(inp_example: dsp.Example) -> dsp.Example:
    return inp_example.copy(text_to_vectorize=inp_example.input)

# create knn index for our training set
dsp.settings.configure(vectorizer=dsp.SentenceTransformersVectorizer())
vectorizer = dsp.settings.vectorizer

In [ ]:
with open('knn_results.txt', 'w') as f:
  num_test = 250
  num_demos = 3
  for i in range(num_test):
    if i % 10 == 0:
      print(i)
    ex = dsp.Example(input=test_short['input'][i], demos=[])
    demos = knn_search(ex, train_dsp, cast_naive_get_only_code, vectorizer, index, k=num_demos)
    ex.demos = demos
    ex, completions = dsp.generate(few_shot_template, n=1, temperature=temperature)(ex, stage='qa')
    _, _ = calc_dist(i, completions, f)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


# CoT

In [ ]:
num = 1
temperature = 0.5
Input = dsp.Type(
    prefix="Input:",
    desc="${the code to be revised}")

Output = dsp.Type(
    prefix="Output:",
    desc="${the revised code}",
    format=dsp.format_answers)

Rationale = dsp.Type(
    prefix="Rationale:",
    desc="${the problems in the provided code}"
)

cot_template = dsp.Template(
    instructions="Revise the code to have better coding style by renaming, removing redundancy, safety checking, etc. Don't include any comments.",
    input=Input(),
    rationale=Rationale(),
    output=Output())

In [ ]:
def calc_dist(i, completions, f=None):
  dist = 0
  same = False
  if f:
    f.write(str(len(completions.data)) + '\n')
  for ind in completions.data:
    if f:
      f.write(ind.output.replace('\n', ''))
      f.write('\n')
    tmp1 = ''.join(ind.output.strip().split())
    tmp2 = ''.join(test_short['output'][i].strip().split())
    d = editdistance.eval(tmp1, tmp2) / len(tmp2)
    dist += d
    if d == 0:
      same = True
  return (dist / num), same

In [ ]:
with open('cot_results_no_rat.txt', 'w') as f:
  num_test = 250
  num_demos = 3
  for i in range(num_test):
    if i % 10 == 0:
      print(i)
    ex = dsp.Example(input=test_short['input'][i], demos=[])
    demos = knn_search(ex, train_dsp, cast_naive_get_only_code, vectorizer, index, k=num_demos)
    ex.demos = demos
    ex, completions = dsp.generate(cot_template, n=1, temperature=temperature)(ex, stage='qa')
    _, _ = calc_dist(i, completions, f)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240


# Eval

In [ ]:
import numpy as np
from dsp.utils import EM, F1
from nltk.translate.bleu_score import sentence_bleu

with open('cot_results_no_rat.txt', 'r') as f:
  i = -1
  total = 0
  l = [0]
  for line in f:
    if len(line.strip()) == 1:
      i += 1
      total += l[0]
      l = []
      continue
    tmp1 = ' '.join(line.strip().split())
    tmp2 = ' '.join(test_short['output'][i].strip().split())
    d4 = editdistance.eval(tmp1, tmp2) / len(tmp2)
    d1 = EM(tmp1, [tmp2])
    d2 = F1(tmp1, [tmp2])
    d3 = sentence_bleu([tmp2.split()], tmp1.split(), weights=(0.25, 0.25, 0.25, 0.25))
    l.append(np.array([d1, d2, d3, d4]))
  print(total / 250)

[0.02       0.55987101 0.26388626 0.58736817]


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
with open('cot_results.txt', 'r') as f:
  i = -1
  total = []
  l = [0]
  for line in f:
    if len(line.strip()) == 1:
      i += 1
      total.append(l[0])
      l = []
      continue
    tmp1 = ' '.join(line.strip().split())
    tmp2 = ' '.join(test['output'][i].strip().split())
    d1 = editdistance.eval(tmp1, tmp2) / len(tmp2)
    d2 = F1(tmp1, [tmp2])
    d3 = EM(tmp1, [tmp2])
    d4 = sentence_bleu([tmp2.split()], tmp1.split(), weights=(0.25, 0.25, 0.25, 0.25))
    d = d1 + d2 + d3 + d4
    l.append(d)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
import numpy as np
from dsp.utils import EM, F1
from nltk.translate.bleu_score import sentence_bleu

with open('cot_results_no_rat.txt', 'r') as f:
  i = -1
  total = []
  l = [0]
  for line in f:
    if len(line.strip()) == 1:
      i += 1
      total.append(l[0])
      l = []
      continue
    tmp1 = ' '.join(line.strip().split())
    tmp2 = ' '.join(test_short['output'][i].strip().split())
    d4 = editdistance.eval(tmp1, tmp2) / len(tmp2)
    d1 = EM(tmp1, [tmp2])
    d2 = F1(tmp1, [tmp2])
    d3 = sentence_bleu([tmp2.split()], tmp1.split(), weights=(0.25, 0.25, 0.25, 0.25))
    d = d1 + d2 + d3 - d4
    l.append(d)
total = np.array(total[1:])


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
np.argsort(total)


array([ 48,  99,  82, 182, 139,  72, 194, 118,  97,  10,  33,  90,   6,
       119,  23,  54, 187, 134,   4, 173,  35,  86, 166,  24,  15, 192,
       135,  13,  62, 153, 141,   2, 178, 177, 167, 123,  38, 242, 179,
        80,  69, 238, 128, 105, 202, 170,  46, 243, 107,  93, 122, 140,
       120,  76, 181,  77, 136,  22,  56,  95, 237, 234, 144, 155, 109,
        65,   5, 158, 164,   7, 150, 198, 207,  31, 169, 193, 143, 218,
        64,  36, 211,  57, 121, 176,  70, 196, 159,  32,   9, 165, 146,
       124, 152, 222,  41, 201, 180,  49, 162,  14,  17, 174, 156,  37,
       216,  25, 172, 129, 117,  94, 213,  51,  79, 104, 125,  89, 138,
       219,  78,  60,  73, 246, 239, 221,  16, 236,  47, 190, 230,  11,
       168,  85,  59, 130, 160,   1, 197,  40, 203, 227, 154, 205, 225,
        55, 148, 200, 229, 232,  26, 244,  98, 163, 108, 151, 100,  88,
       183, 231, 245,  29,  71, 185,  91, 220, 115,  28, 212,  75, 142,
        27, 102,   3,  21,  81,  96, 131,  92,  52, 106, 110, 11

In [ ]:
i = 188
print(test_short['input'][i])
print(test_short['output'][i])

public boolean isAttachable() { return attachable == null ? false : attachable; }
public Boolean isAttachable() { return attachable; }
